# Run this notebook to launch my model in real time with your web-camera.
  - Make sure to set up model_path in the code according to your environment so you can use the state dictionary of my tunned model that you installed on your PC;
  - The name of the dictionary: **extra-version3_resnet50_fc_layer4_layer3_tuned_kritika_data_5classes_1channel-images.pth**

In [ ]:
pip install torch torchvision opencv-python matplotlib



   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.6/159.8 MB 11.8 MB/s eta 0:00:14
   ---------------------------------------- 1.1/159.8 MB 11.7 MB/s eta 0:00:14
   ---------------------------------------- 1.7/159.8 MB 12.1 MB/s eta 0:00:14
    --------------------------------------- 2.2/159.8 MB 11.8 MB/s eta 0:00:14
    --------------------------------------- 2.8/159.8 MB 12.1 MB/s eta 0:00:14
    --------------------------------------- 3.6/159.8 MB 12.0 MB/s eta 0:00:14
   - -------------------------------------- 4.1/159.8 MB 11.9 MB/s eta 0:00:14
   - -------------------------------------- 4.7/159.8 MB 12.0 MB/s eta 0:00:13
   - -------------------------------------- 5.1/159.8 MB 11.6 MB/s eta 0:00:14
   - -------------------------------------- 6.2/159.8 MB 12.7 MB/s eta 0:00:13
   - -------------------------------------- 7.0/159.8 MB 12.9 MB/s eta 0:00:12
   - -------------------------------------- 7.8/159.8 MB 13

In [ ]:
import torch
from torchvision import models
import torch.nn as nn

# Define the model architecture
model = models.resnet50(pretrained=False)
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False) #make sure we have 1-chane as input for conv layer
num_ftrs = model.fc.in_features
num_classes = 5
model.fc = nn.Linear(num_ftrs, num_classes)  # Update this with your number of classes

# Correct path usage with a raw string
model_path = r"C:\Users\kirillnartov\Downloads\extra-version3_resnet50_fc_layer4_layer3_tuned_kritika_data_5classes_1channel-images.pth"
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

model.eval()

# Assuming you're using CPU
device = torch.device('cpu')
model.to(device)

C:\Users\kirillnartov\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\kirillnartov\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
import cv2
from torchvision import transforms
from PIL import Image

transform = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Resize((224, 224)),
    transforms.Resize((270, 270)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

# Load the cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')


In [ ]:
# Class names for predictions
class_names = ['anger', 'fear', 'happy', 'neutral', 'sad']

In [ ]:
import cv2
from PIL import Image
import torch
import numpy as np

# Camera capture setup
cap = cv2.VideoCapture(0)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break

        # Convert to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Extract the face region from the frame
            face_region = gray[y:y+h, x:x+w]

            # Convert face region to a PIL Image
            pil_image = Image.fromarray(face_region)

            # Apply transformations
            face_tensor = transform(pil_image).unsqueeze(0).to(device)

            # Model prediction
            with torch.no_grad():
                output = model(face_tensor)
                _, predicted = torch.max(output, 1)
                predicted_emotion = class_names[predicted.item()]

            # Draw rectangle around the face and annotate predicted emotion
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            cv2.putText(frame, predicted_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

        # Display the resulting frame
        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    cap.release()
    cv2.destroyAllWindows()


KeyboardInterrupt: 